In [32]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
plt.style.use('seaborn')

import yfinance as yf



C:\Users\Himanshu Gupta\AppData\Local\Temp\ipykernel_4100\2010723511.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [33]:
indices=['^VIX','^NYA','^FCHI','^STI','^GDAXI']

equities = ['TJX', 'GOOGL', 'AMZN', 'TSLA', 'SONY']

In [34]:
start_date='2010-01-01'
end_date='2023-05-01'

indices_data = yf.download(indices, start=start_date, end=end_date)['Adj Close']


[*********************100%***********************]  5 of 5 completed


In [35]:

equities_data = yf.download(equities, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [36]:
indices_returns = indices_data.pct_change()
equities_returns = equities_data.pct_change()


In [37]:
indices_cumulative_returns = (1 + indices_returns).cumprod()
equities_cumulative_returns = (1 + equities_returns).cumprod()


In [38]:
print(indices_cumulative_returns)


               ^FCHI    ^GDAXI      ^NYA      ^STI      ^VIX
Date                                                        
2010-01-04       NaN       NaN       NaN       NaN       NaN
2010-01-05  0.999736  0.997282  1.003839  1.008889  0.965569
2010-01-06  1.000922  0.997690  1.006955  1.012416  0.956088
2010-01-07  1.002698  0.995215  1.009171  1.006460  0.951098
2010-01-08  1.007765  0.998233  1.013459  1.009746  0.904691
...              ...       ...       ...       ...       ...
2023-04-24  1.886875  2.622878  2.130103  1.148555  0.842814
2023-04-25  1.876349  2.624230  2.096912  1.138885  0.936128
2023-04-26  1.860168  2.611598  2.078493  1.137970  0.940120
2023-04-27  1.864448  2.612379  2.106208  1.133865  0.849800
2023-04-28  1.866357  2.632538  2.121800  1.129885  0.787425

[3442 rows x 5 columns]


In [39]:
print(equities_cumulative_returns)

                 AMZN     GOOGL      SONY        TJX        TSLA
Date                                                            
2010-01-04        NaN       NaN       NaN        NaN         NaN
2010-01-05   1.005900  0.995596  0.995336   1.026914         NaN
2010-01-06   0.987677  0.970499  0.994337   1.031076         NaN
2010-01-07   0.970874  0.947906  0.992672   1.083796         NaN
2010-01-08   0.997162  0.960543  1.012991   1.069645         NaN
...               ...       ...       ...        ...         ...
2023-04-24  15.864077  6.756380  3.049967  10.365933  102.061514
2023-04-25  15.320388  6.621214  3.023318  10.142853  100.881101
2023-04-26  15.680359  6.612288  3.124917  10.224694   96.536188
2023-04-27  16.403286  6.859667  3.167555  10.420054  100.579722
2023-04-28  15.750559  6.843728  2.987009  10.404213  103.166575

[3353 rows x 5 columns]


In [40]:
def calculate_max_drawdown(data):
    cumulative_returns = (1 + data).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    max_drawdown = drawdown.min()
    return max_drawdown

indices_max_drawdowns = calculate_max_drawdown(indices_returns)
equities_max_drawdowns = calculate_max_drawdown(equities_returns)


In [41]:
print(indices_max_drawdowns)
print(equities_max_drawdowns)

^FCHI    -0.385585
^GDAXI   -0.387794
^NYA     -0.381142
^STI     -0.382211
^VIX     -0.818479
dtype: float64
AMZN    -0.561453
GOOGL   -0.443201
SONY    -0.761869
TJX     -0.425535
TSLA    -0.736322
dtype: float64


In [42]:
risk_free_rate = 0.0  # Set the risk-free rate

def calculate_sharpe_ratio(data, risk_free_rate):
    excess_returns = data - risk_free_rate
    annualized_returns = excess_returns.mean() * 252  # Assuming 252 trading days in a year
    annualized_volatility = data.std() * np.sqrt(252)
    sharpe_ratio = annualized_returns / annualized_volatility
    return sharpe_ratio

def calculate_sortino_ratio(data, risk_free_rate):
    downside_returns = data.copy()
    downside_returns[downside_returns > 0] = 0  # Consider only negative returns for downside
    annualized_returns = data.mean() * 252
    downside_deviation = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (annualized_returns - risk_free_rate) / downside_deviation
    return sortino_ratio

indices_sharpe_ratio = calculate_sharpe_ratio(indices_returns, risk_free_rate)
indices_sortino_ratio = calculate_sortino_ratio(indices_returns,risk_free_rate)


In [43]:
equities_sharpe_ratio=calculate_sharpe_ratio(equities_returns,risk_free_rate)
equities_sortino_ratio=calculate_sortino_ratio(equities_returns,risk_free_rate)

In [44]:
print(equities_sharpe_ratio)
print(equities_sortino_ratio)

AMZN     0.790264
GOOGL    0.665747
SONY     0.415726
TJX      0.813747
TSLA     0.916188
dtype: float64
AMZN     1.339940
GOOGL    1.114266
SONY     0.717438
TJX      1.359297
TSLA     1.588879
dtype: float64


In [45]:
print(indices_sharpe_ratio)
print(indices_sortino_ratio)

^FCHI     0.326903
^GDAXI    0.453788
^NYA      0.405123
^STI      0.133931
^VIX      0.582936
dtype: float64
^FCHI     0.515102
^GDAXI    0.716414
^NYA      0.616927
^STI      0.209181
^VIX      1.260121
dtype: float64
